In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy

In [3]:
# import the data, get into correct format
# set up for visuals
df = pd.read_excel("/Users/avery/OneDrive/Documents/GitHub/Clinical_TLB_2023-2024/lung_cancer_tlb.xlsx")
df['CancerType'] = np.where(df['CancerType'].isna(), 'Control', df['CancerType'])

# get location of cut off values
lower_column_index = df.columns.get_loc("T51")
upper_column_index = df.columns.get_loc("T83.1")
label_column_index = df.columns.get_loc("CancerType")

column_indices = np.arange(lower_column_index, upper_column_index)
column_indices = np.append(column_indices, 0)
column_indices = np.append(column_indices, 1)



column_indices = np.append(column_indices, label_column_index)

df = df.iloc[:, column_indices]


In [4]:
df_boruta = df[(df["CancerType"] == "Control") | (df["CancerType"] == "Adenocarcinoma")]

df_boruta['CancerType'] = np.where(df_boruta["CancerType"] == "Adenocarcinoma", 1, 0)


features = df_boruta.drop(["sample_id", "pub_id", "CancerType"], axis = 1)
features_1 = features.diff(axis = 1).iloc[:, 1:]
features = features.diff(axis = 1).iloc[:, 1:].values

labels = df_boruta["CancerType"].values
labels = labels.ravel()



C:\Users\avery\AppData\Local\Temp\ipykernel_8788\267829100.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_boruta['CancerType'] = np.where(df_boruta["CancerType"] == "Adenocarcinoma", 1, 0)


In [5]:
clf = RandomForestClassifier(n_jobs=-1)
clf.fit(features, labels)


RandomForestClassifier(n_jobs=-1)

In [6]:
feat_selector = BorutaPy(clf, n_estimators='auto', verbose=2, random_state=1)

# find all relevant features
feat_selector.fit(features, labels)

# check selected features
feat_selector.support_

# check ranking of features
feat_selector.ranking_

# call transform() on X to filter it down to selected features
X_filtered = feat_selector.transform(features)


Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	320
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	320
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	320
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	320
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	320
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	320
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	320
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	6
Rejected: 	314
Iteration: 	9 / 100
Confirmed: 	0
Tentative: 	6
Rejected: 	314
Iteration: 	10 / 100
Confirmed: 	0
Tentative: 	6
Rejected: 	314
Iteration: 	11 / 100
Confirmed: 	0
Tentative: 	6
Rejected: 	314
Iteration: 	12 / 100
Confirmed: 	0
Tentative: 	6
Rejected: 	314
Iteration: 	13 / 100
Confirmed: 	0
Tentative: 	6
Rejected: 	314
Iteration: 	14 / 100
Confirmed: 	0
Tentative: 	6
Rejected: 	314
Iteration: 	15 / 100
Confirmed: 	0
Tentative: 	6
Rejected: 	314
Iteration: 	16 / 100
Confirmed: 	0
Tentative: 	6


In [7]:
feat_selector.ranking_


array([267,  65, 135, 224, 132, 214,  23,   2,  48,   4,   6,  37,  83,
        39, 138,  61, 169, 161, 201,  89, 216, 162, 200, 240,  42,  60,
         8,   3,   2,  16,  32,  20,  31,  51,  26,  34,  79, 106, 232,
       119, 155, 249, 226, 101,  25, 134, 120,  41, 104,  92, 107, 224,
       179, 162, 204, 109, 183, 181, 172, 264,  57, 197,  85,  68,  75,
        54,  87, 203, 253,  83, 113,  26,  70,  71,  64,  81,  57,  62,
        72, 212,  90, 126, 114, 245, 141, 313, 155, 206, 216, 312, 215,
       282, 308, 248, 299, 301, 237, 258, 280, 242, 311, 304, 316, 315,
       317, 237, 218, 223, 185, 244, 190, 130,  87,  90, 109,  55, 159,
        46, 130, 186, 169,  67, 209, 206, 242, 126, 198, 117,  55, 272,
       274, 279, 307, 292, 233, 255, 209, 264, 306, 295, 309, 276, 194,
       142, 233, 294, 252, 122, 108, 228,  94, 188, 150, 270,  50,  59,
       175, 206, 169, 189,  30,   1,   7,  43, 103,  95,  20,  14, 112,
        96, 124, 182, 104,  74,  72,  38,  36,  28,  29,  15,  4

In [8]:
selected_features = features_1.columns[feat_selector.support_]

# Extract the corresponding columns from the original DataFrame
features_1[selected_features]

,T67.2,T71.9,T75
3,-0.00216,-0.00357,-0.00176
7,-0.00062,-0.00226,-0.00107
8,-0.00127,-0.00194,-0.00239
9,-0.00033,-0.00190,-0.00138
11,-0.00120,-0.00176,-0.00151
...,...,...,...
194,-0.00182,-0.00387,-0.00125
195,-0.00109,-0.00181,-0.00091
196,-0.00230,-0.00354,-0.00113
197,-0.00110,-0.00385,-0.00154
